# Data Cleaning for Project 3: Superfund Site Data

## Setup

### Import Libraries

In [1]:
import pandas as pd
import os
import json

### Set File Locations

In [2]:
# note that some of the raw data files are very large
# these very large files are located in a gitignored directory.

# raw data files
raw_data_file = "../00_Data/raw_data/priorities_list_full.json"

# clean data files
cleaned_data_csv = "../00_Data/cleaned_data/cleaned_priorities_list.csv"
cleaned_data_json = "../00_Data/cleaned_data/cleaned_priorities_list.json"

## Exploring the Dataset

### Bring in the tract data

In [3]:
data_df = pd.read_json(raw_data_file)

### Take a look at the site data
Some things to note here:
1. There are 1344 records and 359 columns.
2. There are plenty of nulls about.
3. Financial values, e.g. 'Med_House_Value_BG_ACS_09_13', are stored as strings.

In [4]:
data_df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1344 entries, 0 to 1343
Data columns (total 359 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   FIPS_Full                         1344 non-null   int64  
 1   address                           1344 non-null   object 
 2   city                              1344 non-null   object 
 3   date_added                        1344 non-null   object 
 4   federal_facility_ind              1344 non-null   object 
 5   federal_register_url              1344 non-null   object 
 6   geocode_source                    1344 non-null   object 
 7   latitude                          1339 non-null   float64
 8   longitude                         1339 non-null   float64
 9   site_epa_id                       1344 non-null   object 
 10  site_name                         1344 non-null   object 
 11  site_narrative_url                1342 non-null   object 
 12  site_

### Reformat financial data

In [5]:
# take a peek at the financial data stored as strings to see what kind of fromatting we're dealing with.
data_df['Aggr_House_Value_ACS_09_13'].head()

0    $46,888,000
1    $55,911,000
2    $15,265,000
3    $80,646,000
4    $25,773,800
Name: Aggr_House_Value_ACS_09_13, dtype: object

In [6]:
# make a list of the financial format columns
# grab all the columns with 'object' dtype
string_columns = list(data_df.select_dtypes(include=['object']).columns)
string_columns

['address',
 'city',
 'date_added',
 'federal_facility_ind',
 'federal_register_url',
 'geocode_source',
 'site_epa_id',
 'site_name',
 'site_narrative_url',
 'site_progress_url',
 'text',
 'State_name',
 'County_name',
 'Med_HHD_Inc_BG_ACS_09_13',
 'Med_HHD_Inc_BG_ACSMOE_09_13',
 'Med_HHD_Inc_TR_ACS_09_13',
 'Med_HHD_Inc_TR_ACSMOE_09_13',
 'Aggregate_HH_INC_ACS_09_13',
 'Aggregate_HH_INC_ACSMOE_09_13',
 'Med_House_Value_BG_ACS_09_13',
 'Med_House_Value_BG_ACSMOE_09_13',
 'Med_house_value_TR_ACS_09_13',
 'Med_house_value_TR_ACSMOE_09_13',
 'Aggr_House_Value_ACS_09_13',
 'Aggr_House_Value_ACSMOE_09_13',
 'avg_Agg_HH_INC_ACS_09_13',
 'avg_Agg_HH_INC_ACSMOE_09_13',
 'avg_Agg_House_Value_ACS_09_13',
 'avg_Agg_House_Value_ACSMOE_09_13']

In [7]:
# leave off the first 13 entries ('address' ... 'County_name') since we don't need to reformat those
financial_columns = string_columns[13:]

In [8]:
# Loop through and reformat the columns by taking out the '$' and ',', and then changing the dtype to 'float'.
# This can take a minute or two.
for x in financial_columns:
    data_df[[x]] = (data_df[x].replace( '[\$,)]','', regex=True )
                     .replace( '[(]','-',   regex=True ).astype(float))

In [9]:
# Quick check to verify the format.
data_df['Aggr_House_Value_ACS_09_13'].head()

0    46888000.0
1    55911000.0
2    15265000.0
3    80646000.0
4    25773800.0
Name: Aggr_House_Value_ACS_09_13, dtype: float64

### Missing GeoData
Since we are using a map visualization, we are sensitive to missing latitude and longitude.

In [10]:
data_df[data_df['latitude'].isnull()]

,FIPS_Full,address,city,date_added,federal_facility_ind,federal_register_url,geocode_source,latitude,longitude,site_epa_id,...,pct_TEA_MailOutMailBack_CEN_2010,pct_TEA_Update_Leave_CEN_2010,pct_Census_Mail_Returns_CEN_2010,pct_Vacant_CEN_2010,pct_Deletes_CEN_2010,pct_Census_UAA_CEN_2010,pct_Mailback_Count_CEN_2010,pct_FRST_FRMS_CEN_2010,pct_RPLCMNT_FRMS_CEN_2010,pct_BILQ_Mailout_count_CEN_2010
70,0,"RTE 3, JOBOS, PR 00654",Jobos,09/21/1984,No,https://semspub.epa.gov/src/document/11/189627,NaN,NaN,NaN,PRD980763783,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,0,"ROAD 670 KM 3.7 PALO ALTO, MANATI, PR 00701",Manati,04/30/2003,No,http://www.gpo.gov/fdsys/pkg/FR-2003-04-30/pdf...,NaN,NaN,NaN,PRD987367299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,0,"STATE ROAD #2 KM. 19.7 (ACUNA STREET), CANDELE...",Candeleria Ward,02/04/2000,No,http://www.gpo.gov/fdsys/pkg/FR-2000-02-04/pdf...,NaN,NaN,NaN,PRD987376662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,0,"ROAD 674, KM 2, RIO ABAJO WARD, PR 00763",Rio Abajo Ward,07/22/1999,No,http://www.gpo.gov/fdsys/pkg/FR-1999-07-22/pdf...,NaN,NaN,NaN,PRD980512669,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1138,0,NaN,Hudson River,09/21/1984,No,https://semspub.epa.gov/src/document/11/189627,NaN,NaN,NaN,NYD980763841,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Other Cleaning

In [11]:
# convert the column names to lowercase to avoid problems later on.
data_df.columns= data_df.columns.str.lower()

# fill in nulls with zeros
data_df.fillna(0, inplace = True)

# make sure the census block groups and tracts come through as integers rather than floats
data_df.tract = data_df.tract.astype(int)
data_df.block_group = data_df.block_group.astype(int)

# rename the 'text' column to avoid potential confusion
data_df.rename(columns={"text": "site_text"}, inplace = True)

### Select Columns

In [12]:
# select columns we want to keep here
# for now, proof-of-concept
desired_columns = [x.lower() for x in [
                "FIPS_Block_Group",
                "address",
                "city",
                "latitude",
                "longitude",
                "site_score",
                "site_text",
                "State_name",
                "County_name",
                "Tract",
                "Block_Group",
                "Tot_Population_CEN_2010",
                "Hispanic_CEN_2010",
                "NH_Blk_alone_CEN_2010",
                "NH_AIAN_alone_CEN_2010",
                "NH_Asian_alone_CEN_2010",
                "NH_NHOPI_alone_CEN_2010",
                "NH_SOR_alone_CEN_2010",
                "College_ACS_09_13",
                "No_Health_Ins_ACS_09_13",
                "Med_HHD_Inc_BG_ACS_09_13",
                "Aggregate_HH_INC_ACS_09_13",
                "Tot_Vacant_Units_CEN_2010",
                "Renter_Occp_HU_CEN_2010",
                "Owner_Occp_HU_CEN_2010",
                "No_Plumb_ACS_09_13",
                "Med_House_Value_BG_ACS_09_13",
                "pct_Hispanic_CEN_2010",
                "pct_NH_Blk_alone_CEN_2010",
                "pct_NH_AIAN_alone_CEN_2010",
                "pct_NH_Asian_alone_CEN_2010",
                "pct_NH_NHOPI_alone_CEN_2010",
                "pct_NH_SOR_alone_CEN_2010",
                "pct_Not_HS_Grad_ACS_09_13",
                "pct_No_Health_Ins_ACS_09_13",
                "pct_Vacant_Units_CEN_2010",
                "pct_Renter_Occp_HU_CEN_2010",
                "pct_Owner_Occp_HU_CEN_2010",
                "pct_No_Plumb_ACS_09_13"]]
cleaned_data_df = data_df[desired_columns]

In [13]:
# last check
cleaned_data_df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1344 entries, 0 to 1343
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   fips_block_group              1344 non-null   int64  
 1   address                       1344 non-null   object 
 2   city                          1344 non-null   object 
 3   latitude                      1344 non-null   float64
 4   longitude                     1344 non-null   float64
 5   site_score                    1344 non-null   float64
 6   site_text                     1344 non-null   object 
 7   state_name                    1344 non-null   object 
 8   county_name                   1344 non-null   object 
 9   tract                         1344 non-null   int32  
 10  block_group                   1344 non-null   int32  
 11  tot_population_cen_2010       1344 non-null   float64
 12  hispanic_cen_2010             1344 non-null   float64
 13  nh_

## Export

In [14]:
# Export to csv
cleaned_data_df.to_csv(cleaned_data_csv, index = False)

In [15]:
# Export to flat json
cleaned_data_df.to_json(cleaned_data_json, orient='records')